In [1]:
!pip install -q -U transformers datasets bitsandbytes peft

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [3]:
import huggingface_hub

huggingface_hub.login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset
dataset = load_dataset("BLACKBUN/old_korean_newspaper_1897_1910_economy_politic_qa")

README.md:   0%|          | 0.00/410 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.46M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8946 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/284 [00:00<?, ? examples/s]

In [5]:
# for 2 billion model

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch

BASE_MODEL = "google/gemma-2-2b-it"
ADAPTER = "jia6776/korean_history_1897_1910_gemma2_2b_lora"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto"  # This will automatically distribute the model across available GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# Load the LoRA adapter
model = PeftModel.from_pretrained(model, ADAPTER)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.2M [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

BASE_MODEL = "google/gemma-2-9b-it"
ADAPTER = "BLACKBUN/korean_history_1897_1910_gemma2_9b_lora_q4"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"  # This will automatically distribute the model across available GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# Load the LoRA adapter
model = PeftModel.from_pretrained(model, ADAPTER)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/81.1M [00:00<?, ?B/s]

In [23]:
question = dataset['test']['Question'][149]
answer = dataset['test']['Answer'][149]

In [24]:
print(f"Q:{question}")

Q:대한제국 시기의 관보에서 언급된 '의정부'와 '궁내부'의 역할은 무엇인가요?


In [25]:
messages = [
    # {
    #     "role": "system",
    #     "content": "You are a helpful assistant."
    # },
    {
        "role": "user",
        "content": f"{question}"
    },

]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [26]:
input_text = prompt
inputs = tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=256)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [27]:
print(f"Model Answer: {generated_text}\n")
print(f"Ground Answer: {answer}")

Model Answer: user
대한제국 시기의 관보에서 언급된 '의정부'와 '궁내부'의 역할은 무엇인가요?
model
'의정부'는 정부의 정책을 수행하고, '궁내부'는 황실과 관련된 행정을 담당하는 부서로, 두 부서 모두 정부의 중앙에서 중요한 역할을 수행했습니다. 의정부는 정부의 정책을 논의하고, 궁내부는 황실의 행정을 감독하며, 두 부서가 협력하여 국가의 운영을 보조하는 역할을 했습니다.

Ground Answer: 대한제국 시기의 관보에서 '의정부'는 국가의 정치적 의사결정을 담당하는 기관으로, 정부의 정책과 법률을 논의하고 결정하는 역할을 했습니다. '궁내부'는 황실과 관련된 업무를 수행하며, 궁중의 행정과 의전 등을 관리하는 기관으로, 황제의 권위를 지키고 궁중의 질서를 유지하는 역할을 했습니다. 이 두 기관은 서로 협력하여 국가 운영에 중요한 역할을 하였으며, 관보의 내용은 이러한 기관들이 어떻게 상호작용하며 국가의 중요한 사안을 처리했는지를 보여줍니다.
